<a href="https://colab.research.google.com/github/opjns/Wildfire-prediction/blob/main/Wildfire_Prediction_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Climate Conditions

##Data

In [ ]:
import kagglehub
import pandas as pd
import os

# Download both datasets
path1 = kagglehub.dataset_download("ayoubjadouli/morocco-wildfire-predictions-2010-2022-ml-dataset")
path2 = kagglehub.dataset_download("amin05/wildfire-analysis-and-prediction")

def load_all_data(path, n_rows=10000):
    dataframes = []
    for file in os.listdir(path):
        full_path = os.path.join(path, file)
        if file.endswith(".parquet"):
            df = pd.read_parquet(full_path)
            dataframes.append(df)
        elif file.endswith(".csv"):
            df = pd.read_csv(full_path)
            dataframes.append(df)
    return dataframes


dfs1 = load_all_data(path1, n_rows=10000)
dfs2 = load_all_data(path2, n_rows=10000)

# Combine
combined_df = pd.concat(dfs1 + dfs2, ignore_index=True)

df = combined_df.copy()

df_filtered = combined_df[['latitude','longitude','NDVI','average_temperature_lag_1', 'precipitation_lag_1','dew_point_lag_1','wind_speed_lag_1','is_fire',]].rename(columns={
    'average_temperature_lag_1': 'Temperature',
    'precipitation_lag_1': 'Precipitation',
    'dew_point_lag_1': 'humidity',
    'wind_speed_lag_1': 'windspeed'})

In [ ]:
print(df_filtered.columns)

Index(['latitude', 'longitude', 'NDVI', 'Temperature', 'Precipitation',
       'humidity', 'windspeed', 'is_fire'],
      dtype='object')


##Train Model

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Assuming df_filtered already exists and has only the relevant columns

# 1. Drop any rows with missing values
df_filtered = df_filtered.dropna()

# 2. Define features and target
X = df_filtered.drop(columns=['is_fire'])  # features
y = df_filtered['is_fire']                 # target

# 3. Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 4. Train a Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 5. Predict on test set
y_pred = model.predict(X_test)

# 6. Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9992884580404241
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     93556
         1.0       1.00      1.00      1.00     93362

    accuracy                           1.00    186918
   macro avg       1.00      1.00      1.00    186918
weighted avg       1.00      1.00      1.00    186918



In [ ]:
print(df['is_fire'].value_counts())

is_fire
1.0    467293
0.0    467293
Name: count, dtype: int64


#Satellite Images

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abdelghaniaaba/wildfire-prediction-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/wildfire-prediction-dataset
